# Build vocab

## Gemini

### Chuẩn bị mô hình

In [ ]:
!pip install google-generativeai -qq

In [ ]:
"""
Install the Google AI Python SDK

$ pip install google-generativeai

See the getting started guide for more information:
https://ai.google.dev/gemini-api/docs/get-started/python
"""

import os
import re
import csv
import google.generativeai as genai

genai.configure(api_key="")

# Create the model
# See https://ai.google.dev/api/python/google/generativeai/GenerativeModel
generation_config = {
  "temperature": 0.5,
  "top_k": 1,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-pro-latest",
  generation_config=generation_config,
  # safety_settings = Adjust safety settings
  # See https://ai.google.dev/gemini-api/docs/safety-settings
)


### Hàm xử lý

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer

# Đảm bảo bạn đã cài đặt NLTK và tải về các gói cần thiết
nltk.download('punkt_tab')

def split_long_sentence_to_sentence_pair(long_sentence:str)-> tuple[str, str]:
    if len(long_sentence.split()) > 40:
        words = word_tokenize(long_sentence)
        midpoint = len(words) // 3

        # Tìm vị trí hợp lý để tách câu tại dấu câu gần nhất hoặc khoảng trắng
        for i in range(midpoint, len(words)):
            if words[i] in [',', ';', '.', '?', '!']:
                sentence1 = words[:i+1]
                sentence2 = words[i+1:]
                break
            if words[i].lower() in ['which', 'where', 'that', 'it', 'who', 'how']:
                sentence1 = words[:i]
                sentence2 = words[i:]
                break
        else:
            sentence1 = words[:midpoint]
            sentence2 = words[midpoint:]

        # Ghép lại từ danh sách từ thành câu hoàn chỉnh
        detokenizer = TreebankWordDetokenizer()
        sentence1 = detokenizer.detokenize(sentence1)
        sentence2 = detokenizer.detokenize(sentence2)

        return [sentence1, sentence2]
    else:
        return [long_sentence]

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
def sepereate_sentences(text:str)->str:

    # Các dấu chấm để phân biệt giữa các câu
    dau_cham = ['.', '!', '?']

    # Tách các câu dựa trên dấu chấm
    sentences = []
    current_sentence = []

    for char in text:
        current_sentence.append(char)
        if char in dau_cham:
            sentences.append(''.join(current_sentence).strip())
            current_sentence = []

    # Kiểm tra nếu còn câu cuối cùng không có dấu chấm
    if current_sentence:
        sentences.append(''.join(current_sentence).strip())
    return " ".join(sentences)

In [ ]:
import nltk
nltk.download('punkt')

def split_paragraph_into_sentences(paragraph:str)->list[str]:
    sentences = nltk.sent_tokenize(paragraph)
    sentence_list = []
    for sentence in sentences:
        # if len(sentence.split()) > 38:
        #     sentence1, sentence2 = split_long_sentence_to_sentence_pair(sentence)
        #     sentence_list.append(sentence1)
        #     sentence_list.append(sentence2)
        # else:
            sentence_list.append(sentence)

    print(f"Number of sentences: {len(sentence_list)}")
    print("Sentences:", sentences)

    return sentence_list

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def extract_level_3(vocab_list, parent_level=1):
    """
    Extract all vocab with level 3 from a nested vocab structure
    """
    words = []
    for vocab in vocab_list:
        if isinstance(vocab, dict) and vocab['level'] == parent_level and 'child' in vocab:
            words.extend(extract_level_3(vocab['child'], parent_level + 1))
        elif isinstance(vocab, dict) and vocab['level'] == 3:
            words.append(vocab)
    return words


def update_vocab_para_with_dataframe(vocab_para, df):
    """
    Update vocab_para with values from a DataFrame
    """
    def update_nested_vocab(vocab_list, df, parent_level=1):
        for vocab in vocab_list:
            if isinstance(vocab, dict) and vocab['level'] == parent_level and 'child' in vocab:
                update_nested_vocab(vocab['child'], df, parent_level + 1)
            elif isinstance(vocab, dict) and vocab['level'] == 3:
                word_row = df[df['value'] == vocab['value']]

    for item in vocab_para:
        update_nested_vocab(item['vocab'], df)


### PROMPT

In [ ]:
EXAMPLES = """Here's some examples:
<example_1>
<Input>
<paragraph> Whenever John is thinking about his next vacation, he is filled with excitement. Watching documentaries about exotic destinations has always fascinated him, and it encourages him to plan adventurous trips. However, his financial constraints often prevent him from embarking on these journeys, which he finds quite frustrating </paragraph>
<sentence> Whenever John is thinking about his next vacation, he is filled with excitement. </sentence>
</Input>
<Output>
1|Whenever|Whenever|Conjunction|Bất cứ khi nào|/wɛnˈɛvər/|Trong đoạn văn, "whenever" được sử dụng để diễn tả ý nghĩa "bất cứ lúc nào" hoặc "mỗi khi".|whenever + clause|Whenever I go to the beach, I collect seashells. (Bất cứ khi nào tôi đến bãi biển, tôi đều nhặt vỏ sò.)|Whenever she feels stressed, she listens to music. (Mỗi khi cô ấy cảm thấy căng thẳng, cô ấy nghe nhạc.)
2|John|John|Noun|John|/dʒɒn/|Trong đoạn văn, "John" được sử dụng như một tên riêng, chỉ người đàn ông có tên là John.|None|John is a doctor. (John là một bác sĩ.)|John loves to travel to new places. (John thích đi du lịch đến những nơi mới.)
3|is|be|verb (auxiliary verb)|đang|/bi:/|Trong đoạn văn, "is" là một phần của cấu trúc hiện tại tiếp diễn, diễn tả một hành động đang xảy ra hoặc một trạng thái hiện tại, cụ thể là hỗ trợ cho "thinking".|None|He is reading a book. (Anh ấy đang đọc sách.)|She is happy today. (Hôm nay cô ấy vui.)
4|thinking|think|verb (transitive verb)|nghĩ|/ˈθɪŋk/|Trong đoạn văn, "thinking" được sử dụng để diễn tả hành động suy nghĩ, cân nhắc trong tâm trí.|think about something/someone|She is thinking about her future. (Cô ấy đang nghĩ về tương lai của mình.)|I am thinking about quitting my job. (Tôi đang nghĩ đến việc nghỉ việc.)
5|about|about|preposition|về|/əˈbaʊt/|Trong đoạn văn, "about" được sử dụng để chỉ đối tượng hoặc chủ đề mà ai đó đang nghĩ đến.|how/what about + noun/noun phrase/pronoun|What about dinner tonight? (Còn bữa tối thì sao?)|What about him? (Còn anh ấy thì sao?)
6|his|his|determiner|của anh ấy|/hɪz/|Trong đoạn văn, "his" tham chiếu đến "John", bổ nghĩa cho danh từ "next vacation".|None|This is his car. (Đây là xe của anh ấy.)|His favorite color is blue. (Màu yêu thích của anh ấy là màu xanh.)
7|next|next|adjective|tiếp theo|/nɛkst/|Trong đoạn văn, "next" được sử dụng để chỉ kỳ nghỉ tiếp theo của John.|None|The next bus will arrive in 10 minutes. (Chuyến xe buýt tiếp theo sẽ đến sau 10 phút nữa.)|We will meet again next week. (Chúng ta sẽ gặp lại nhau vào tuần sau.)
8|vacation|vacation|noun|kỳ nghỉ|/veɪˈkeɪʃən/|Trong đoạn văn, "vacation" được sử dụng để chỉ một khoảng thời gian nghỉ ngơi, thư giãn, thường là đi du lịch.|go on vacation|We are planning a vacation to the beach. (Chúng tôi đang lên kế hoạch cho một kỳ nghỉ ở bãi biển.)|They went on vacation to Europe last year. (Họ đã đi nghỉ ở Châu Âu vào năm ngoái.)
9|he|he|pronoun|anh ấy|/hiː/|Trong đoạn văn, "he" tham chiếu đến "John", chủ ngữ của mệnh đề phụ.|None|He is a teacher. (Anh ấy là một giáo viên.)|He told me that he would be late. (Anh ấy nói với tôi rằng anh ấy sẽ đến muộn.)
10|is|be|verb (auxiliary verb)|trở nên|/bi:/|Trong đoạn văn, "is" được dùng để nối chủ ngữ "he" với trạng từ "filled" diễn tả trạng thái của John.|None|He is happy. (Anh ấy hạnh phúc.)|She is tired. (Cô ấy mệt mỏi.)
11|filled|fill|verb (transitive verb)|tràn đầy|/fɪl/|Trong đoạn văn, "filled" được sử dụng để diễn tả trạng thái John cảm thấy tràn đầy excitement khi nghĩ về kỳ nghỉ tiếp theo.|fill something with something|She filled the page with writing. (Cô viết đầy trang giấy.)|Bags filled with household waste. (Túi chứa đầy rác thải sinh hoạt.)
12|with|with|preposition|với|/wɪθ/|Trong đoạn văn, "with" được sử dụng để chỉ thứ mà John cảm thấy tràn đầy, đó là "excitement".|None|I like coffee with milk. (Tôi thích cà phê sữa.)|She is living with her parents. (Cô ấy đang sống với bố mẹ.)
13|excitement|excitement|noun|sự hào hứng|/ɪkˈsaɪtmənt/|Trong đoạn văn, "excitement" được sử dụng để chỉ cảm xúc hào hứng, phấn khích của John khi nghĩ về kỳ nghỉ.|None|The announcement was met with excitement. (Thông báo đã được đón nhận với sự hào hứng.)|Children were jumping with excitement. (Bọn trẻ nhảy lên vì phấn khích.)
</Output>
</example_1>
<example_2>
<Input>
<paragraph> Whenever John is thinking about his next vacation, he is filled with excitement. Watching documentaries about exotic destinations has always fascinated him, and it encourages him to plan adventurous trips. However, his financial constraints often prevent him from embarking on these journeys, which he finds quite frustrating </paragraph>
<sentence> Watching documentaries about exotic destinations has always fascinated him, and it encourages him to plan adventurous trips. </sentence>
</Input>
<Output>
1|Watching|watch|Noun (gerund)|việc xem|/ˈwɒtʃ/|Trong đoạn văn, "watching" được sử dụng như danh động từ, đóng vai trò là chủ ngữ của mệnh đề, miêu tả hoạt động xem phim tài liệu.|watch something|Watching documentaries about endangered species motivates me to support conservation efforts. (Việc xem phim tài liệu về các loài động vật có nguy cơ tuyệt chủng thúc đẩy tôi hỗ trợ các nỗ lực bảo tồn.)|John enjoys watching movies in his free time. (John thích xem phim vào thời gian rảnh.)
2|documentaries|documentary|noun|phim tài liệu|/ˌdɒkjuˈmentəri/|Trong đoạn văn, "documentaries" được sử dụng để chỉ thể loại phim dựa trên thực tế, cung cấp thông tin về một chủ đề cụ thể.|make a documentary|The film crew traveled to Africa to make a documentary about wildlife conservation. (Đoàn làm phim tới Châu Phi để làm phim tài liệu về bảo tồn động vật hoang dã.)|She learned a lot from watching documentaries about history. (Cô ấy đã học được rất nhiều từ việc xem phim tài liệu về lịch sử.)
3|about|about|preposition|về|/əˈbaʊt/|Trong đoạn văn, "about" được sử dụng để chỉ chủ đề được đề cập đến trong phim tài liệu.|about something|The documentary is about the life cycle of butterflies. (Bộ phim tài liệu nói về vòng đời của loài bướm.)|We were talking about our plans for the summer. (Chúng tôi đang nói về kế hoạch cho mùa hè của mình.)
4|exotic|exotic|adjective|kỳ lạ|/ɪɡˈzɒtɪk/|Trong đoạn văn, "exotic" được sử dụng để miêu tả điểm đến độc đáo, khác thường và hấp dẫn.|exotic location|She dreamed of traveling to exotic islands with pristine beaches. (Cô ấy mơ ước được đi du lịch đến những hòn đảo kỳ lạ với những bãi biển hoang sơ.)|The market was filled with exotic fruits and spices. (Chợ đầy ắp những loại trái cây và gia vị kỳ lạ.)
5|destinations|destination|noun|điểm đến|/ˌdestɪˈneɪʃən/|Trong đoạn văn, "destinations" được sử dụng để chỉ những nơi mà John muốn đến du lịch.|travel destination|Paris is a popular travel destination for tourists from around the world. (Paris là một điểm đến du lịch nổi tiếng đối với khách du lịch từ khắp nơi trên thế giới.)|They chose Paris as their honeymoon destination. (Họ đã chọn Paris là điểm đến cho tuần trăng mật của mình.)
6|has|have|verb (auxiliary verb)|None|/hæv/|Trong đoạn văn, "has" được sử dụng như trợ động từ trong thì hiện tại hoàn thành, diễn tả một hành động đã xảy ra trong quá khứ và kéo dài đến hiện tại.|None|He has always loved to travel. (Anh ấy luôn thích đi du lịch.)|She has finished her homework. (Cô ấy đã hoàn thành bài tập về nhà của mình.)
7|always|always|adverb|luôn luôn|/ˈɔːlweɪz/|Trong đoạn văn, "always" được sử dụng để nhấn mạnh việc John luôn bị cuốn hút bởi việc xem phim tài liệu về các điểm đến kỳ lạ.|None|He has always been fascinated by ancient civilizations. (Anh ấy luôn bị cuốn hút bởi các nền văn minh cổ đại.)|She always arrives on time. (Cô ấy luôn đến đúng giờ.)
8|fascinated|fascinate|verb (transitive verb)|thu hút|/ˈfæsɪneɪt/|Trong đoạn văn, "fascinated" được sử dụng như một động từ ở dạng quá khứ phân từ trong cấu trúc bị động, diễn tả John bị thu hút bởi việc xem phim tài liệu.|be fascinated by something|The children were fascinated by the magician's tricks. (Những đứa trẻ đã bị thu hút bởi những trò ảo thuật của nhà ảo thuật.)|The story of her life fascinated me. (Câu chuyện về cuộc đời cô ấy đã thu hút tôi.)
9|him|him|pronoun|anh ấy|/hɪm/|Trong đoạn văn, "him" tham chiếu đến "John", tân ngữ của động từ "fascinated".|None|John loves to travel, and he often plans his trips months in advance. (John thích đi du lịch, và anh ấy thường lên kế hoạch cho chuyến đi của mình trước vài tháng.)|John is a doctor. He works in a hospital. (John là bác sĩ. Anh ấy làm việc trong một bệnh viện.)
10|and|and|conjunction|và|/ænd/|Trong đoạn văn, "and" được sử dụng để nối hai mệnh đề, diễn tả mối quan hệ liên quan giữa việc John thích xem phim tài liệu và việc anh ấy muốn đi du lịch.|None|He likes to read books, and he also enjoys watching movies. (Anh ấy thích đọc sách, và anh ấy cũng thích xem phim.)|She went to the store, and she bought some milk. (Cô ấy đã đến cửa hàng, và cô ấy đã mua một ít sữa.)
11|it|it|pronoun|nó|/ɪt/|Trong đoạn văn, "it" được sử dụng để thay thế cho việc xem phim tài liệu, tránh lặp lại chủ ngữ.|None|Watching documentaries is a great way to learn about the world, and it can inspire you to travel. (Xem phim tài liệu là một cách tuyệt vời để tìm hiểu về thế giới, và nó có thể truyền cảm hứng cho bạn đi du lịch.)|The book is on the table. It is a gift from my friend. (Cuốn sách ở trên bàn. Nó là một món quà từ bạn của tôi.)
12|encourages|encourage|verb (transitive verb)|khuyến khích|/ɪnˈkʌrɪdʒ/|Trong đoạn văn, "encourages" được sử dụng để diễn tả việc xem phim tài liệu thúc đẩy John lên kế hoạch cho những chuyến đi mạo hiểm.|encourage someone to do something|My parents always encouraged me to pursue my dreams. (Bố mẹ tôi luôn khuyến khích tôi theo đuổi ước mơ của mình.)|The teacher encouraged her students to ask questions. (Giáo viên khuyến khích học sinh đặt câu hỏi.)
13|him|him|pronoun|anh ấy|/hɪm/|Trong đoạn văn, "him" tham chiếu đến "John", tân ngữ của động từ "encourages".|None|John loves to travel, and he often plans his trips months in advance. (John thích đi du lịch, và anh ấy thường lên kế hoạch cho chuyến đi của mình trước vài tháng.)|John is a doctor. He works in a hospital. (John là bác sĩ. Anh ấy làm việc trong một bệnh viện.)
14|to|to|preposition|để|/tuː/|Trong đoạn văn, "to" được sử dụng để chỉ mục đích của việc xem phim tài liệu là khuyến khích John lên kế hoạch cho những chuyến đi mạo hiểm.|None|She went to the store to buy some milk. (Cô ấy đã đến cửa hàng để mua một ít sữa.)|He studied hard to pass the exam. (Anh ấy đã học tập chăm chỉ để vượt qua kỳ thi.)
15|plan|plan|verb (transitive verb)|lên kế hoạch|/plæn/|Trong đoạn văn, "plan" được sử dụng để diễn tả việc John lên kế hoạch cho những chuyến đi mạo hiểm.|plan something|They are planning a trip to Europe next summer. (Họ đang lên kế hoạch cho một chuyến đi đến Châu Âu vào mùa hè tới.)|She needs to plan her time carefully. (Cô ấy cần phải lên kế hoạch thời gian cẩn thận.)
16|adventurous|adventurous|adjective|phiêu lưu|/ədˈventʃərəs/|Trong đoạn văn, "adventurous" được sử dụng để miêu tả tính chất mạo hiểm, đầy thử thách của những chuyến đi mà John muốn thực hiện.|adventurous trip|We are planning an adventurous trip to the Amazon rainforest next summer. (Chúng tôi đang lên kế hoạch cho một chuyến đi mạo hiểm đến rừng nhiệt đới Amazon vào mùa hè tới.)|She has an adventurous spirit and loves to try new things. (Cô ấy có một tinh thần thích phiêu lưu và thích thử những điều mới.)
17|trips|trip|noun|chuyến đi|/trɪp/|Trong đoạn văn, "trips" được sử dụng để chỉ những chuyến du lịch, thường là đến những nơi xa và thú vị.|go on a trip|They went on a road trip along the coast. (Họ đã thực hiện một chuyến đi dọc bờ biển.)|We are planning a trip to the mountains. (Chúng tôi đang lên kế hoạch cho một chuyến đi đến vùng núi.)
</Output>
</example_2>
<example_3>
<Input>
<paragraph> Whenever John is thinking about his next vacation, he is filled with excitement. Watching documentaries about exotic destinations has always fascinated him, and it encourages him to plan adventurous trips. However, his financial constraints often prevent him from embarking on these journeys, which he finds quite frustrating. </paragraph>
<sentence> However, his financial constraints often prevent him from embarking on these journeys, which he finds quite frustrating. </sentence>
</Input>
<Output>
1|However|however|adverb|tuy nhiên|/ˌhaʊˈevər/|Trong đoạn văn, "however" được sử dụng để thể hiện sự tương phản với ý trước đó.|None|John wanted to go to Europe. However, he couldn't afford it. (John muốn đi du lịch châu Âu. Tuy nhiên, anh ấy không đủ tiền.)|The weather was bad. However, we still had a great time. (Thời tiết rất xấu. Tuy nhiên, chúng tôi vẫn có khoảng thời gian tuyệt vời.)
2|his|his|determiner|của anh ấy|/hɪz/|Trong đoạn văn, "his" tham chiếu đến "John", bổ nghĩa cho danh từ "financial constraints".|None|His favorite color is blue. (Màu yêu thích của anh ấy là màu xanh lam.)|He lost his keys yesterday. (Hôm qua anh ấy đã đánh mất chìa khóa của mình.)
3|financial|financial|adjective|tài chính|/faɪˈnænʃl/|Trong đoạn văn, "financial" được sử dụng để miêu tả những điều liên quan đến tiền bạc.|financial situation|They are working hard to improve their financial situation. (Họ đang làm việc chăm chỉ để cải thiện tình hình tài chính của họ.)|The company is in a strong financial position. (Công ty đang ở trong một vị thế tài chính mạnh mẽ.)
4|constraints|constraint|noun|sự ràng buộc|/kənˈstreɪnt/|Trong đoạn văn, "constraints" được sử dụng để chỉ những giới hạn, đặc biệt là về mặt tài chính.|constraints of time/money/space|Due to constraints of time, we had to cut the meeting short. (Do hạn chế về thời gian nên chúng tôi phải cắt ngắn cuộc họp.)|The company's growth is constrained by the lack of skilled workers. (Sự tăng trưởng của công ty bị hạn chế bởi việc thiếu công nhân lành nghề.)
5|often|often|adverb|thường|/ˈɒfn/|Trong đoạn văn, "often" được sử dụng để chỉ sự thường xuyên xảy ra của một hành động.|None|He often goes to the gym after work. (Anh ấy thường đến phòng tập thể dục sau giờ làm việc.)|We often eat out on weekends. (Chúng tôi thường đi ăn ngoài vào cuối tuần.)
6|prevents|prevent|verb (transitive verb)|ngăn cản|/prɪˈvent/|Trong đoạn văn, "prevent" được sử dụng để chỉ việc ngăn chặn ai đó làm điều gì đó.|prevent somebody/something from doing something|The heavy rain prevented us from going out. (Trận mưa lớn đã ngăn cản chúng tôi đi ra ngoài.)|Wearing a seatbelt can prevent serious injuries in a car accident. (Việc đeo dây an toàn có thể ngăn ngừa những chấn thương nghiêm trọng trong một vụ tai nạn xe hơi.)
7|him|him|pronoun|anh ấy|/hɪm/|Trong đoạn văn, "him" tham chiếu đến "John", tân ngữ của động từ "prevent".|None|I gave him the book. (Tôi đã đưa cho anh ấy cuốn sách.)|She asked him to help her with the groceries. (Cô ấy đã yêu cầu anh ấy giúp cô ấy xách đồ tạp hóa.)
8|from|from|preposition|từ|/frəm/|Trong đoạn văn, "from" được sử dụng để chỉ điểm bắt đầu của một hành động hoặc trạng thái.|None|He is traveling from London to Paris. (Anh ấy đang đi du lịch từ Luân Đôn đến Paris.)|She borrowed a book from the library. (Cô ấy đã mượn một cuốn sách từ thư viện.)
9|embarking|embark|verb (gerund)|bắt đầu|/ɪmˈbɑːrk/|Trong đoạn văn, "embarking" được sử dụng để chỉ việc bắt đầu một hành động, thường là một hành trình hoặc một dự án mới.|embark on something|She is about to embark on a new career. (Cô ấy sắp bắt đầu một sự nghiệp mới.)|The company is embarking on a major expansion program. (Công ty đang bắt đầu một chương trình mở rộng lớn.)
10|on|on|preposition|trên|/ɒn/|Trong đoạn văn, "on" được sử dụng để chỉ vị trí hoặc trạng thái của một đối tượng.|None|The book is on the table. (Cuốn sách nằm trên bàn.)|He is on vacation. (Anh ấy đang đi nghỉ.)
11|these|these|determiner|những|/ðiːz/|Trong đoạn văn, "these" được sử dụng để chỉ những chuyến đi phiêu lưu mà John muốn thực hiện.|None|These are the books I bought yesterday. (Đây là những cuốn sách tôi đã mua hôm qua.)|These shoes are too small for me. (Những đôi giày này quá nhỏ so với tôi.)
12|journeys|journey|noun|chuyến đi|/ˈdʒɜːrni/|Trong đoạn văn, "journeys" được sử dụng để chỉ những chuyến đi dài, thường là đến những nơi xa xôi.|go on a journey|They went on a long journey across the country. (Họ đã thực hiện một chuyến đi dài xuyên quốc gia.)|The journey to success is often long and difficult. (Hành trình đến thành công thường dài và khó khăn.)
13|which|which|pronoun|mà|/wɪtʃ/|Trong đoạn văn, "which" được sử dụng để thay thế cho mệnh đề "his financial constraints often prevent him from embarking on these journeys" và làm tân ngữ cho động từ "finds".|None|He bought a new car, which he had always wanted. (Anh ấy đã mua một chiếc xe hơi mới, thứ mà anh ấy luôn mong muốn.)|She studied hard for the exam, which she passed with flying colors. (Cô ấy đã học tập chăm chỉ cho kỳ thi, kỳ thi mà cô ấy đã vượt qua một cách xuất sắc.)
14|he|he|pronoun|anh ấy|/hiː/|Trong đoạn văn, "he" được sử dụng để chỉ John.|None|He is a doctor. (Anh ấy là một bác sĩ.)|He loves to play soccer. (Anh ấy thích chơi bóng đá.)
15|finds|find|verb (transitive verb)|cảm thấy|/faɪnd/|Trong đoạn văn, "finds" được sử dụng để chỉ cảm xúc hoặc suy nghĩ của John về việc bị hạn chế tài chính.|find something + adjective|He finds the movie very interesting. (Anh ấy thấy bộ phim rất thú vị.)|She finds it difficult to wake up early in the morning. (Cô ấy thấy khó khăn khi thức dậy sớm vào buổi sáng.)
16|quite|quite|adverb|khá|/kwaɪt/|Trong đoạn văn, "quite" được sử dụng để chỉ mức độ, ở đây là mức độ frustrating.|None|The food is quite good. (Món ăn khá ngon.)|It's quite cold outside today. (Hôm nay trời khá lạnh.)
17|frustrating|frustrating|adjective|thất vọng|/frʌˈstreɪtɪŋ/|Trong đoạn văn, "frustrating" được sử dụng để miêu tả cảm xúc của John khi anh ấy không thể thực hiện những chuyến đi do hạn chế về tài chính.|None|It's frustrating when you can't find what you're looking for. (Thật bực bội khi bạn không thể tìm thấy thứ mình đang tìm kiếm.)|The traffic jam was so frustrating! (Tắc đường thật bực bội!)
</Output>
</example_3>
"""
TASK_DESCRIPTION = f"""<Instruction>
Proceed these steps with the above input:
- Split sentence into words list, then go through each word in the list and give me the following information:
+ Word : Give me a word (original word as in a sentence)
+ Word display: Give me the lemmatization form only. For example, if the word is “changed”, you just need to display “change”. If the word is like “clocks”, you just need to show “clock” for me.
+ Word Class: Only give me its word class in the context of the paragraph/passage. Do not capitalize. If the word is a verb, the verb type of that word must be clearly indicated it may be in list: Phrase verb, (Intransitive Verbs, transitive Verbs), (verb, linking verb, Auxiliary Verbs, model verb)
+ Vietnamese Meaning: Only give me the Vietnamese meaning of the word in the context of paragraph above. If the word doesn’t have a meaning in Vietnamese, give me "None".
+ IPA: IPA of the lemmatization word (format: /..../). Not of the word which are splited from sentence.
+ Vietnamese Explanation: Explain it in Vietnamese in the context of this paragraph/passage in less than 60 Vietnamese word segments. Follow this format: Trong đoạn văn, "the english word" được sử dụng để....
+ Verb Structure/ Collocation: If the word is a verb, provide the verb structure (example: the word pattern of the verb “provide” is “provide somebody with something/ provide something for somebody”). If the word is not a verb, provide the most common collocation of the word.
+ Example_1: Write a sentence with that word and match the meaning with above paragraph to help an intermediate level student. If there is a verb structure or collocation of that word, try to give an example that uses that verb structure or collocation. Translate the example into Vietnamese and put it in ()
+ Example_2: Write a sentence with that word and match the context with above paragraph to help an intermediate level student. If there is a verb structure or collocation of that word, try to give an example that uses that verb structure or collocation. Translate the example into Vietnamese and put it in ()
*Notes:
- Before performing the above steps, you need to determine where the sentence I provide is located in the paragraph, especially when determining the reference of words, you must check the sentences before and after my provided sentence
- When you slit sentence into words list, note the following: don't include punctuation and numbers; word must be a complete word it cludes compound words (Ex: Fairy-tale, ...), not a part of a word; Note the abbreviations to deduce the original word
- Give me answer similar to the following format in <Output> tag and give me only text in <Output> tag not including the tag <output> at the end. Because I need to save your answer into file csv.
- Notes for "word class": Steps to Identify "-ing" Forms: Determine the Role (Identify if the "-ing" word is a noun, verb, or adjective), Check Verb Phrase, Context Clues (Use context to determine if it describes a noun (adjective) or is part of continuous tenses/passive construction (present/past participle))
- Notes for 'Vietnamese Meaning': if word is in ['it', 'that', 'this', 'his', 'him', ...] then in a sentence, there are some word just used to refer something in the paragraph (To do this you need to locate the sentence I provided). For example, in this sentence: 'Gemini is an AI model. It is also used to handle difficult situations in our life. Moreover, Gemini 1.5-pro is the most intelligent model. It can handle those situations that the normal model can not do'. I want you to know that the word "It" in 'It is also used to handle difficult situations' refers to 'Gemini' and the second 'It' in the sentence  'It can handle those situations that the normal model can not do' refers to 'Gemini 1.5-pro'.
- If you find a colaction or verb structure of the word, please give an example using that colaction or verb structure.
- Must have both example_1 and example_2
- Each line of your answer must have the same number of columns as in the example tag
- The IPA will be the word you put in the word display column
- Each word must have 10 values ​​as shown in the example in the output tag
</Instruction>"""

TONE_CONTEXT = f"""You are an expert in explaining vocabulary (in Vietnamese) based on the context of a paragraph or a passage.
Based on the passage and the sentence, proceed with these tasks in <Instruction> tag with the above input.
Please note that do not generate other words outside the passage I provide you."""

OUTPUT_FORMATTING = """Give me answer similar to the following format in <Output> tag and give me only text in <Output> tag. Because I need to save your answer into file csv."""


In [ ]:
import time
from random import random
import pandas as pd
import random
import time
from tqdm import tqdm
import psycopg2
import json

def save_to_db(vocab_list):
    try:
        # Kết nối đến database
        with psycopg2.connect(
            dbname="",
            user="",
            password="",
            host="",
            port=""
        ) as conn:
            with conn.cursor() as cursor:
                # Câu lệnh chèn dữ liệu
                insert_query = """
                INSERT INTO public.vocab
                (vocab_id, value, word_display, word_class, meaning, ipa, explanation, collocation, verb_structure, example)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                ON CONFLICT (vocab_id) DO NOTHING;
                """

                for vocab in vocab_list:
                    try:
                        # Xử lý ký tự đặc biệt trong ví dụ để tránh lỗi SQL
                        sanitized_examples = [e.replace('"', '\"') for e in vocab.get('example', [])]

                        # Chuyển example thành kiểu _text[] PostgreSQL
                        example = '{' + ', '.join(f'"{e}"' for e in sanitized_examples) + '}'

                        # Debug dữ liệu
                        print(f"Inserting data: {vocab['vocab_id']} - Word Display: {vocab['word_display']} - Example: {example}")

                        # Chèn dữ liệu
                        cursor.execute(insert_query, (
                            vocab['vocab_id'],
                            vocab['value'],
                            vocab['word_display'],
                            vocab['word_class'],
                            vocab['meaning'],
                            vocab['ipa'],
                            vocab['explanation'],
                            vocab['collocation'],
                            vocab.get('verb_structure', 'None'),
                            example  # Lưu Example dạng _text[]
                        ))

                    except Exception as e:
                        # Rollback giao dịch lỗi để tiếp tục
                        conn.rollback()
                        print(f"Error inserting data: {vocab}")
                        print(f"Error details: {e}")

            # Commit dữ liệu
            conn.commit()

    except Exception as e:
        print(f"Database Connection Error: {e}")

def get_current_word_count():
    try:
        # Kết nối đến database
        with psycopg2.connect(
            dbname="",
            user="",
            password="",
            host="",
            port=""
        ) as conn:
            with conn.cursor() as cursor:
                # Truy vấn để đếm số lượng từ hiện tại
                cursor.execute("SELECT COUNT(*) FROM public.vocab;")
                result = cursor.fetchone()
                return result[0]  # Trả về số lượng từ
    except Exception as e:
        print(f"Database Error: {e}")
        return 0

def get_max_sentence_index(quiz_id: int):
    try:
        with psycopg2.connect(
            dbname="",
            user="",
            password="",
            host="",
            port=""
        ) as conn:
            with conn.cursor() as cursor:
                cursor.execute(
                    "SELECT MAX(SPLIT_PART(vocab_id, '_', 2)::INTEGER) FROM public.vocab WHERE SPLIT_PART(vocab_id, '_', 1)::INTEGER = %s;",
                    (quiz_id,)
                )
                result = cursor.fetchone()
                return result[0] if result[0] is not None else 0
    except Exception as e:
        print(f"Database Error: {e}")
        return 0


def build_vocab(passage: str, id_passage: str, quiz_id: int):
    word_id = get_current_word_count() + 1
    vocab_list = []  # Danh sách từ vựng để lưu vào DB
    paragraph_list = passage.split('\n')  # Chia đoạn văn thành các đoạn nhỏ

    sentence_index = get_max_sentence_index(quiz_id)


    for index_para, paragraph in enumerate(paragraph_list):
        # Tách đoạn văn thành câu
        cleared_paragraph = sepereate_sentences(paragraph)
        sentences = split_paragraph_into_sentences(cleared_paragraph)

        for index_sent, sentence in enumerate(sentences):
            # Increment sentence index only if it is a new full sentence (ending with a period)
            if sentence.strip().endswith('.'):
                sentence_index += 1  # Increment sentence index

            sentence_split = split_long_sentence_to_sentence_pair(sentence)
            split_part_index = 0  # Tracks parts after splitting

            for sent in sentence_split:
                split_part_index += 1  # Increment split part index
                TASK_CONTEXT = f""" Below is a paragraph/passage related from IELTS Reading materials:
<Input>
<paragraph> {passage} </paragraph>
<sentence> {sent} </sentence>
</Input>"""

                PROMPT = ""
                if TONE_CONTEXT:
                    PROMPT += f"\n{TONE_CONTEXT}"
                if TASK_CONTEXT:
                    PROMPT += f"\n{TASK_CONTEXT}"
                if TASK_DESCRIPTION:
                    PROMPT += f"\n{TASK_DESCRIPTION}"
                if OUTPUT_FORMATTING:
                    PROMPT += f"\n{OUTPUT_FORMATTING}"
                if EXAMPLES:
                    PROMPT += f"\n{EXAMPLES}"

                retry_count = 0
                max_retries = 3
                delay = random.randint(1, 5)

                while retry_count < max_retries:
                    try:
                        chat_session = model.start_chat(history=[])
                        response = chat_session.send_message(PROMPT)

                        output = response.text
                        output = output.replace('<Output>\n', '').replace('</Output>', '').replace('```\n', '').replace('\n```', '')
                        print(output)

                        csv_lines = output.splitlines()
                        word_index = 0  # Reset word index for each new split sentence

                        for line in csv_lines:
                            columns = line.split('|')

                            if len(columns) < 10:
                                print(f"Skipped line (invalid columns): {line}")
                                continue

                            try:
                                word_index += 1  # Increment word index for each word

                                # Create vocab_id based on quizID_sentenceIndex_wordIndex
                                vocab_id = f"{quiz_id}_{sentence_index}_{word_index}"
                                value = columns[1].strip()
                                word_display = columns[2].strip()
                                word_class = columns[3].strip()
                                meaning = columns[4].strip()
                                ipa = columns[5].strip()
                                explanation = columns[6].strip()
                                collocation = columns[7].strip()

                                example_1 = columns[8].strip() if columns[8].strip() else '-'
                                example_2 = columns[9].strip() if columns[9].strip() else '-'
                                example = [example_1, example_2]

                                print(f"Processed vocab: {vocab_id}, Value: {value}, Example: {example}")

                                if value == '-' and word_class == '-' and meaning == '-':
                                    continue

                                vocab = {
                                    'vocab_id': vocab_id,
                                    'value': value,
                                    'word_display': word_display,
                                    'word_class': word_class,
                                    'meaning': meaning,
                                    'ipa': ipa,
                                    'explanation': explanation,
                                    'collocation': collocation,
                                    'example': example
                                }

                                vocab_list.append(vocab)
                                save_to_db([vocab])

                            except Exception as e:
                                print(f"Parsing Error: {e}")

                        break

                    except Exception as e:
                        print(f"Error: {e}")
                        time.sleep(delay)
                        retry_count += 1
                time.sleep(random.randint(1, 5))
    return {'quiz_id': quiz_id, 'vocab': vocab_list}


### Run

##### Vocab

In [ ]:
input = """F. The other major casualty of the "naturalism" of environmental scientists has been the indigenous Amazonians, whose habits of hunting, fishing, and slash-and-burn cultivation often have been represented as harmful to the habitat. In the clash between environmentalists and developers, the Indians, whose presence is in fact crucial to the survival of the forest, have suffered the most. The new understanding of the pre-history of Amazonia, however, points toward a middle ground. Archaeology makes clear that with judicious management selected parts of the region could support more people than anyone thought before. The long-buried past, it seems, offers hope for the future."""

In [ ]:
result = build_vocab(input, str(1), quiz_id=2)

Number of sentences: 5
Sentences: ['F. The other major casualty of the "naturalism" of environmental scientists has been the indigenous Amazonians, whose habits of hunting, fishing, and slash-and-burn cultivation often have been represented as harmful to the habitat.', 'In the clash between environmentalists and developers, the Indians, whose presence is in fact crucial to the survival of the forest, have suffered the most.', 'The new understanding of the pre-history of Amazonia, however, points toward a middle ground.', 'Archaeology makes clear that with judicious management selected parts of the region could support more people than anyone thought before.', 'The long-buried past, it seems, offers hope for the future.']
1|The|the|determiner|cái, những|/ðiː/|Trong đoạn văn, "the" được dùng để xác định danh từ "other major casualty".|None|The cat sat on the mat. (Con mèo ngồi trên tấm thảm.)|The sun is shining brightly today. (Mặt trời hôm nay chiếu sáng rực rỡ.)
2|other|other|adjective

In [ ]:
def clean_and_reindex_vocab(quiz_id: int):
    try:
        # Kết nối đến cơ sở dữ liệu
        conn = psycopg2.connect(
            dbname="",
            user="",
            password="",
            host="",
            port=""
        )
        cursor = conn.cursor()

        # Xóa các bản ghi không hợp lệ trong quiz_id đã chỉ định
        delete_query = """
        DELETE FROM public.vocab
        WHERE ((value = '-' AND word_class = '-' AND meaning = '-' AND ipa = '-')
           OR (value = 'None' AND word_class = 'None' AND meaning = 'None' AND ipa = 'None'))
          AND SPLIT_PART(vocab_id, '_', 1)::INTEGER = %s;
        """
        cursor.execute(delete_query, (quiz_id,))
        print("Deleted rows with invalid values ('-', 'None') for quiz_id:", quiz_id)

        # Truy vấn các bản ghi hợp lệ của quiz_id đã chỉ định, sắp xếp theo id và vocab_id
        select_query = """
        SELECT id, vocab_id FROM public.vocab
        WHERE SPLIT_PART(vocab_id, '_', 1)::INTEGER = %s
        ORDER BY SPLIT_PART(vocab_id, '_', 2)::INTEGER ASC, SPLIT_PART(vocab_id, '_', 3)::INTEGER ASC;
        """
        cursor.execute(select_query, (quiz_id,))
        rows = cursor.fetchall()

        # Đánh lại số vocab_id trong phạm vi câu
        update_query = """
        UPDATE public.vocab
        SET vocab_id = %s
        WHERE id = %s;
        """

        current_sentence = -1
        word_index = 0
        for row in rows:
            db_id = row[0]
            old_vocab_id = row[1]

            # Lấy chỉ số câu và từ cũ
            parts = old_vocab_id.split('_')
            sentence_index = int(parts[1])

            # Nếu là câu mới, reset word_index
            if sentence_index != current_sentence:
                current_sentence = sentence_index
                word_index = 1
            else:
                word_index += 1

            # Tạo vocab_id mới
            new_vocab_id = f"{quiz_id}_{sentence_index}_{word_index}"
            cursor.execute(update_query, (new_vocab_id, db_id))
            print(f"Updated {old_vocab_id} -> {new_vocab_id}")

        # Lưu thay đổi
        conn.commit()
        print("Reindexing completed successfully for quiz_id:", quiz_id)

        # Đóng kết nối
        cursor.close()
        conn.close()

    except Exception as e:
        print(f"Error: {e}")

# Chạy hàm reindex
clean_and_reindex_vocab(quiz_id=2)

Deleted rows with invalid values ('-', 'None') for quiz_id: 2
Updated 2_1_1 -> 2_1_1
Updated 2_1_2 -> 2_1_2
Updated 2_1_3 -> 2_1_3
Updated 2_1_4 -> 2_1_4
Updated 2_1_5 -> 2_1_5
Updated 2_1_6 -> 2_1_6
Updated 2_1_7 -> 2_1_7
Updated 2_1_8 -> 2_1_8
Updated 2_1_9 -> 2_1_9
Updated 2_1_10 -> 2_1_10
Updated 2_1_11 -> 2_1_11
Updated 2_1_12 -> 2_1_12
Updated 2_1_13 -> 2_1_13
Updated 2_1_14 -> 2_1_14
Updated 2_1_15 -> 2_1_15
Updated 2_1_16 -> 2_1_16
Updated 2_1_17 -> 2_1_17
Updated 2_1_18 -> 2_1_18
Updated 2_1_19 -> 2_1_19
Updated 2_1_20 -> 2_1_20
Updated 2_1_21 -> 2_1_21
Updated 2_1_22 -> 2_1_22
Updated 2_1_23 -> 2_1_23
Updated 2_1_24 -> 2_1_24
Updated 2_1_25 -> 2_1_25
Updated 2_1_26 -> 2_1_26
Updated 2_1_27 -> 2_1_27
Updated 2_1_28 -> 2_1_28
Updated 2_1_29 -> 2_1_29
Updated 2_1_30 -> 2_1_30
Updated 2_1_31 -> 2_1_31
Updated 2_2_1 -> 2_2_1
Updated 2_2_2 -> 2_2_2
Updated 2_2_3 -> 2_2_3
Updated 2_2_4 -> 2_2_4
Updated 2_2_5 -> 2_2_5
Updated 2_2_6 -> 2_2_6
Updated 2_2_7 -> 2_2_7
Updated 2_2_8 -> 2_2

In [ ]:
import psycopg2

# Hàm để chuyển dữ liệu từ database hiện tại sang database khác
def migrate_vocab_to_another_db():
    try:
        # Kết nối đến database nguồn
        source_conn = psycopg2.connect(
            dbname="",
            user="",
            password="",
            host="",
            port=""
        )
        source_cursor = source_conn.cursor()

        # Kết nối đến database đích
        target_conn = psycopg2.connect(
            dbname="",
            user="",
            password="",
            host="",
            port=""
        )
        target_cursor = target_conn.cursor()

        # Lấy tất cả dữ liệu từ bảng nguồn
        source_cursor.execute("SELECT * FROM public.temp_vocab;")
        rows = source_cursor.fetchall()

        # Chèn dữ liệu vào bảng đích
        insert_query = """
        INSERT INTO public.vocab (
            id, vocab_id, value, word_display, word_class, meaning, ipa, explanation, collocation, verb_structure, example
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (id) DO NOTHING;
        """

        for row in rows:
            target_cursor.execute(insert_query, row)

        # Lưu thay đổi ở database đích
        target_conn.commit()
        print("Data migration completed successfully!")

        # Đóng kết nối
        source_cursor.close()
        source_conn.close()
        target_cursor.close()
        target_conn.close()

    except Exception as e:
        print(f"Error: {e}")


migrate_vocab_to_another_db()

Data migration completed successfully!
